<a href="https://colab.research.google.com/github/miladmirzazadeh/Gamein-Prototype/blob/master/Prototype_with_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [17]:
import numpy as np 
import pandas as pd 
import random

class Parameters():
  ###################### reading all dataframes from google drive ...... to call them faster and easier in runtime 

  land_stockhouse_df = pd.read_excel("/content/drive/My Drive/projects/gamein_prototype/parameters.xlsx",sheet_name="land_stockhouse", header=0,index_col=0)
  land_machine_df = pd.read_excel("/content/drive/My Drive/projects/gamein_prototype/parameters.xlsx",sheet_name="land_machine", header=0,index_col=0)
  machines_info_df = pd.read_excel("/content/drive/My Drive/projects/gamein_prototype/parameters.xlsx",sheet_name="machines_info", header=0,index_col=0)
  products_volume_df = pd.read_excel("/content/drive/My Drive/projects/gamein_prototype/parameters.xlsx",sheet_name="products_info", header=0,index_col=0)
  bom_df = pd.read_excel("/content/drive/My Drive/projects/gamein_prototype/parameters.xlsx",sheet_name="bom", header=0,index_col=0)
  gamein_buy_price = pd.read_excel("/content/drive/My Drive/projects/gamein_prototype/parameters.xlsx",sheet_name="gamein_buy_price", header=0,index_col=0) #milad : added .... price of products that gamein buys
  gamein_sell_price = pd.read_excel("/content/drive/My Drive/projects/gamein_prototype/parameters.xlsx",sheet_name="gamein_sell_price", header=0,index_col=0)

class Player():
  players_name=[]

  def __init__(self, player_name):
    self.name = player_name
    #remove from available
    self.cash = 10000000 # بودجه اولیه
    self.land_objects=dict()

  def calculate_final_net_worth(self):
    #to calculate final net worth
    # inventory_value_sum=sum([self.land_objects[land].calculate_inventory_value() for land in list(self.land_objects.keys())]) #nima jaygozine 3 khate bala
    # net_worth = inventory_value_sum + self.cash 
    net_worth = self.cash #incomplete
    return net_worth 

  def add_land(self, land_id):
    self.land_objects.update({land_id : Land(land_id)}) # یه اینستنس از لند میسازه که ورودی بهش آیدی میدیم . بعد توی خود لند ، بر حسب آیدی بقیه فیچر های لند رو ست میکنیم
    

  def add_machine(self, land_id, machine_id):
    self.land_objects[land_id].add_machine(machine_id)
class Gamein_Bazaar():
  def put_into_market():
    pass
class Land():
  lands_list = list(Parameters.land_machine_df.index) #amirreza یه دیتا فریم که یه آی دی زمین داره و ماشین
  available_lands= lands_list.copy() #آی دی های باقی مانده
  
  def __init__(self, land_id):
    self.machine = _
    self.id = land_id
    self.cap = Parameters.land_stockhouse_df.loc[self.id, "cap"]
    self.stockhouse = StockHouse(self.cap) # اینجا دیگه دیکشنری نیست چون فرض بر اینه که یه دونه انبار بیشتر نداریم 


    # self.stockhouse = StockHouse()
  def add_machine(self, machine_id):
    self.machine= Machine(machine_id)
class StockHouse():
  def __init__(self, cap):
    self.total_cap = cap
    self.remaining_cap = self.total_cap
    self.inventory_df = pd.DataFrame([], columns= ["total_inventory", "inhand_inventory"]) # indexes = product names 
    # self.cost_per_product = 100

  def add(self, product_name, amount, just_total=False, just_inhand=False): #milad : just_total va just_inhand ezafe shod
    volume = amount * Product.products_volume.loc[product_name, 'volume'] 
    if just_inhand:
      if (not (product_name in self.inventory_df.index)) or (pd.isnull(self.inventory_df.loc[product_name,"inhand_inventory"])) : #milad : agar inhand_inventory == nan -> chizi behesh ezafe nemishe
        self.inventory_df.loc[product_name,"inhand_inventory"] = amount 
      else:
        self.inventory_df.loc[product_name, "inhand_inventory"] += amount 

    elif just_total:
      if (not (product_name in self.inventory_df.index)) or (pd.isnull(self.inventory_df.loc[product_name,"total_inventory"])):
        self.inventory_df.loc[product_name, "total_inventory"] = amount 
      else:
        self.inventory_df.loc[product_name, "total_inventory"] += amount 
      self.remaining_cap -= volume

    elif not (just_total or just_inhand):# both are False
      if (product_name in self.inventory_df.index):
        self.inventory_df.loc[product_name, ["total_inventory", "inhand_inventory"]] += amount 
        if (pd.isnull(self.inventory_df.loc[product_name,"inhand_inventory"])):
          self.inventory_df.loc[product_name,"inhand_inventory"] = amount 

      else:
        self.inventory_df.loc[product_name, ["total_inventory", "inhand_inventory"]] = amount 
      self.remaining_cap -= volume
        
      
      

  def remove(self, product_name, amount, just_total=False, just_inhand=False):
    volume = amount * Product.products_volume.loc[product_name, 'volume'] 
    if not (just_total or just_inhand): # both are False
      self.inventory_df.loc[product_name, ["total_inventory", "inhand_inventory"]] -= amount 
      self.remaining_cap += volume 

    if (just_total): #total==True, inhand==False
      self.inventory_df.loc[product_name, ["total_inventory"]] -= amount 
      self.remaining_cap += volume 

    if (just_inhand): 
      self.inventory_df.loc[product_name, ["inhand_inventory"]] -= amount
class Machine():

  def __init__(self,machine_id):
    self.id = machine_id
    self.product_name = Parameters.machines_info_df.loc[self.id, "product"]
    # self.price = _ 
    self.enabled = True # False until machine comes :)
    self.working = 0 # False while mahcine works
    self.remaining_production = 0 # reduce production rate per turn , 
    self.production_cost_per_unit = Parameters.machines_info_df.loc[self.id, "production_cost_per_unit"]
    self.fixed_production_cost = Parameters.machines_info_df.loc[self.id, "fixed_production_cost"]
    self.production_rate = Parameters.machines_info_df.loc[self.id, "production_rate"]
    
  def start_production(self, amount):#milad : ezafe shod
    self.working = 1 
    self.remaining_production = amount
    

#nima mitune nabashe
class Product():

  products_bom = Parameters.bom_df
  products_volume = Parameters.products_volume_df # its a df not a series . so use loc[index, "volume"]  



  # def __init__(self):
  #   self.material_needed = 10
  #   self.lower_band = 1
  #   self.upper_band = 20
class Material():
  Prices = _ #dict (id , price)
class Market(): #nima alan avale kar 

  vitrin = pd.DataFrame([], columns=["seller_name","product_name", "amount", "price"]) #milad : seller_obj replaced with seller_name !!! more rational 

  def reset_vitrin():
    Market.vitrin = pd.DataFrame([], columns=["seller_name","product_name", "amount", "price"])

class Controller(): 
  # We have to set all initial values here ...... why ? because we wanna reset them everytime we build Controller class 
  Market.reset_vitrin() 
  Land.available_lands = Land.lands_list.copy()
  #milad : added 
    #adding gamein supply products in Market.vitrin at the first 
  vitrin_id = 1
  for product_name, row in Parameters.gamein_sell_price.iterrows():
    Market.vitrin.loc[vitrin_id] = ["gamein", product_name, 1000000, row.price]
    vitrin_id += 1

  return_message=""
  turn = 0 
  last_turn = 50
  player_objects = dict() #name,object

  def start():  
    Market.reset_vitrin() 
    Land.available_lands = Land.lands_list.copy()
       #milad : added 
    #adding gamein supply products in Market.vitrin at the first 
    vitrin_id = 1
    for product_name, row in Parameters.gamein_sell_price.iterrows():
      Market.vitrin.loc[vitrin_id] = ["gamein", product_name, 1000000, row.price]
      vitrin_id += 1

    Controller.return_message = ""
    Controller.turn = 0 
    Controller.last_turn = 50
    Controller.player_objects = dict() #name,object

  def next_turn():
    Controller.turn+=1 
    #milad : ezafe shod
    for player in list(Controller.player_objects.values()): #continue production
      machine = list(player.land_objects.values())[0].machine
      stockhouse = list(player.land_objects.values())[0].stockhouse
      if machine.working == 1 : 
        if machine.remaining_production <= machine.production_rate:
          stockhouse.add(machine.product_name, amount=machine.remaining_production, just_inhand=True)
          machine.remaining_production = 0
          machine.working = 0 #end production
          print(player.name +"عملیات تولید تمام شد")
          #CommandAnalyzer.show_message(player_name, "عملیات تولید تمام شد")
        else :
          machine.remaining_production -= machine.production_rate
          stockhouse.add(machine.product_name, amount=machine.production_rate, just_inhand=True)

    if Controller.turn == Controller.last_turn : 
      score_board = Controller.score_board()
      Controller.return_message = "End Game. Thank u " + score_board

  
  def score_board():
    net_worths = pd.Series([])
    for player in list(Controller.player_objects.keys()): 
      obj = Controller.player_objects[player] 
      obj.calculate_final_net_worth()
      net_worths[player] = obj 
    net_worths = net_worths.sort(ascending=False)
    return net_worths    

  def create_player(player_name):

    #milad : user's telegram id IS Not his player_name anymore
    #milad : every player can choose a name ..... in CommandAnalyzer we need a dict of telegram username and player name 
    
    if player_name in list(Controller.player_objects.keys()): 
      return ("این نام کاربری قبلا ثبت شده")
    if len(Land.available_lands) == 0 :
      return ("متاسفانه ظرفیت بازی تکمیل شده ") 
    land_id = random.choice(Land.available_lands) #random از بین لیست زمین های در دسترس
    Land.available_lands.remove(land_id)   #remove from available lands
    if player_name == "milad":
      land_id = 30001
    machine_id = Parameters.land_machine_df.loc[land_id, "machine_id"] # از دیتافریم زمین و ماشین خوانده میشود

    Controller.player_objects.update({player_name : Player(player_name)})
    Controller.buy_land(player_name, land_id)
    Controller.buy_machine(player_name, land_id, machine_id)
    machine_name = Parameters.machines_info_df.loc[machine_id, "machine_name"]#milad added
    return ("  با موفقیت وارد بازی شدید . ماشین  " + machine_name +"  به شما تعلق گرفت :)   ")
    
  def buy_land(player_name, land_id):
    Controller.player_objects[player_name].add_land(land_id)
    # Controller.player_objects[name].cash -=         اینجا باید پول زمین رو از حسابش کم کنیم . فعلا لازم نیست  چون خودمون بهش زمین رو اختصاص میدیم 

  def buy_machine(player_name, land_id, machine_id):
    Controller.player_objects[player_name].add_machine(land_id, machine_id)

  def show_status(player_name):#milad : in ro ezafe kardam 
    player_obj = Controller.player_objects[player_name]
    stockhouse = list(player_obj.land_objects.values())[0].stockhouse
    machine = list(player_obj.land_objects.values())[0].machine
    return ({"cash":player_obj.cash, "inventory" : stockhouse.inventory_df, "cap" :stockhouse.remaining_cap, "remaining_production":machine.remaining_production}) # we should consider this type of return in CommandAnalyzer()

  def start_production(player_name, product_name, amount):# This Func should take machine_id too (Later Inshallah)
  
    player_obj = Controller.player_objects[player_name]
    stockhouse = list(player_obj.land_objects.values())[0].stockhouse
    machine = list(player_obj.land_objects.values())[0].machine
    materials = Product.products_bom.loc[product_name]

    cost = machine.fixed_production_cost + machine.production_cost_per_unit * amount

    volume = Product.products_volume.loc[product_name, "volume"] * amount 


    if not (machine.enabled==True and machine.working==0): ### checking if the required machine is available (free/bought)
      return ('ماشین قادر به تولید نیست')
    if product_name != machine.product_name: ### checking existence
      return (" شما ماشین " + product_name + " را ندارید " ) 
    if player_obj.cash < cost :  ############## Checking Player Cash
      return ("پولت کمه ")
    if stockhouse.remaining_cap < volume: ############### Checking Inventory Remaining Cap
      return ("انبارت اینقدر جا نداره  ")
  
    shortage = ""
    for mat, number in materials.iteritems():
      if number > 0 :
        if not (mat in stockhouse.inventory_df.index):
          shortage += str (number*amount)+ " تا " + mat + " .لازم داری که تو انبار موجود نیست " + "\n" + "موجودی قابل استفاده  :  " + "0" +"\n"
        elif (number*amount > stockhouse.inventory_df.loc[mat, "inhand_inventory"]):
          shortage += str (number*amount)+ " تا " + mat + " .لازم داری که تو انبار موجود نیست " + "\n" + "موجودی قابل استفاده  :  " + str(stockhouse.inventory_df.loc[mat, "inhand_inventory"]) +"\n" 
        else :
          stockhouse.remove(mat, number*amount)
    if shortage !="" :
      return (shortage)
    ##################################### when u get here , everything is OK
    player_obj.cash -= cost
    stockhouse.add(product_name, amount, just_total=True)#milad : bayad mogheye shoroo e tolid be total_inventory ezafe she ama be inhand_inventory na..... yani az remaining cap kam she ke ta vaghti tolid mikone natoone anbaro por kone 
    machine.start_production(amount) #milad : ezafe shod 
    return ("تولید با موفقیت آغاز شد") 
    
  
  def sell_to_gamein(player_name, product_name, amount):#milad : foroosh be gamein ham vase mahsool miani hast ham nahaei . price nemigire. 
    if not (product_name in stockhouse_obj.inventory_df.index):#check the inventory
      return ("محصول مورد نظر موجود نیست")
    if stockhouse_obj.inventory_df.loc[product_name, "inhand_inventory"] < amount : 
      return ("این تعداد از محصول مورد نظر موجود نیست ")
    player_obj = Controller.player_objects[player_name]
    stockhouse_obj = list(player_obj.land_objects.values())[0].stockhouse
    player_obj.cash += amount*Parameters.gamein_buy_price.loc[product_name,"price"]
    stockhouse_obj.remove(product_name, amount)
    return ("فروخته شد")
       
 
  def put_into_market(player_name, product_name, amount, price):# it doesn't get stockhouse_id or etc. until we have more than one stockhouse
    player_obj = Controller.player_objects[player_name]
    product_on_vitrin_id = random.choice([i for i in range(100) if not(i in Market.vitrin.index)])   # a random unique id , that will assign to vitrin objects  #nima nemifahmam
    stockhouse_obj = list(player_obj.land_objects.values())[0].stockhouse

    #check the inventory
    if not (product_name in stockhouse_obj.inventory_df.index):
      return ("محصول مورد نظر موجود نیست")
    if stockhouse_obj.inventory_df.loc[product_name, "inhand_inventory"] < amount : 
      return ("این تعداد از محصور مورد نظر موجود نیست ")
    
    existing_row = Market.vitrin[np.logical_and(np.logical_and(Market.vitrin.product_name == product_name, Market.vitrin.price == price), Market.vitrin.seller_name == player_name)]
    if len(existing_row) != 0:
      # add to existing row IF player_name, product_name and price are equal 
      Market.vitrin.loc[existing_row.index, "amount"] += amount
    else : 
      Market.vitrin.loc[product_on_vitrin_id] = [player_name, product_name, amount, price]
    stockhouse_obj.remove(product_name, amount, just_inhand=True)
    return ("تو ویترین قرار گرفت :)")
    
  
  def upgrade_stockhouse(player_name, land_id):
    pass

  def show_product_market_info():
    vitrin = Market.vitrin.copy()
    vitrin["id"] = vitrin.index
    vitrin = vitrin[["id", "product_name" , "amount", "price", "seller_name"]]
    return vitrin


  def buy_product(player_name, product_on_vitrin_id, amount): # آیدی محصول داخل ویترین رو میده که بریم از روی دیکشنری توی مارکت آبجکتش رو پیدا کنیم و انتقال رو انجام بدیم 

    buyer_player = Controller.player_objects[player_name]
    buyer_stockhouse = list(buyer_player.land_objects.values())[0].stockhouse
    product_on_vitrin = Market.vitrin.loc[product_on_vitrin_id] 
    volume = Product.products_volume.loc[product_on_vitrin.product_name, 'volume']
    
    if amount*volume > buyer_stockhouse.remaining_cap: 
      return ("فضای انبار کافی نیست")
    if buyer_player.cash < product_on_vitrin.price * amount :
      return ("پول کافی نیست")
    if amount > product_on_vitrin.amount :
      return ("این مقدار کالا در بازار نیست") 
  
    buyer_player.cash -= amount * product_on_vitrin.price #update cash
    buyer_stockhouse.add(product_on_vitrin.product_name, amount) #updata buyer inventory

    if product_on_vitrin.seller_name != "gamein" : #milad : added 
    #update seller inventory :
      seller_player = Controller.player_objects[product_on_vitrin.seller_name]
      seller_stockhouse = list(seller_player.land_objects.values())[0].stockhouse
      seller_stockhouse.remove(product_on_vitrin.product_name, amount, just_total=True) 
      seller_player.cash += amount * product_on_vitrin.price # add cash to seller player
      product_on_vitrin.amount -= amount

    #update vitrin
    
    if product_on_vitrin.amount == 0 : #### deleting vitrin row ir amount == 0 
      Market.vitrin.drop(product_on_vitrin_id, inplace=True) 

    return ("معامله انجام شد")

In [9]:
!pip install python-telegram-bot

In [22]:
from telegram import InlineQueryResultArticle, InputTextMessageContent, InlineKeyboardButton, InlineKeyboardMarkup, Bot
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackQueryHandler, ConversationHandler
import logging




class CommandAnalyzer():
  bot = _ 
  player_ca_objects = dict() #objects of command analyzer for each player
  playerid_chatid = dict()
  def new_message(update, context, callback =False):
    if callback :
      player_id = update.callback_query.from_user.username
    else : 
      player_id = update.message.from_user.username
    
    if player_id not in CommandAnalyzer.player_ca_objects:
      CommandAnalyzer.player_ca_objects[player_id] = CommandAnalyzer(player_id)
      CommandAnalyzer.playerid_chatid[player_id] = update.message.chat_id
    player_obj = CommandAnalyzer.player_ca_objects[player_id]
    player_obj.update = update
    player_obj.callback = callback 
    
    if callback : 
      player_obj.new_text_message(update.callback_query.data)
      player_obj.message_id = update.callback_query.message.message_id
    else: 
      player_obj.new_text_message(update.message.text)
      player_obj.message_id = update.message.message_id
    

  def new_callback(update,context):
    CommandAnalyzer.new_message(update,context,callback=True)

  def __init__(self,player_id):
    self.callback = False
    self.update = _
    self.player_id = player_id
    self.request = 0
    self.state = 0 
    self.arguments = []
    self.bot = _
    self.chat_id = _
    self.message_id=_

  def show_message(self, text, reply_markup = None):
    CommandAnalyzer.bot.send_message(chat_id = self.chat_id, text = text, reply_markup = reply_markup , message_id = self.message_id)
  def edit_message_text(self, message, reply_markup =None):
    try :
      if self.callback:
        self.update.callback_query.edit_message_text(message, reply_markup = reply_markup)
      else : 
        self.update.message.edit_text(message, reply_markup = reply_markup)
    except :
      self.show_message("ظاهرا ورودی اشتباهی به سیستم داده شده")
    
      
    # print("reply_text_______________________ahsdf;ahdsf;ajshfdasdf")
    # if self.callback: 
    #   self.update.callback_query.edit_message_text(message, reply_markup=reply_markup)
    # else : 
    #   self.update.message.edit_message_text(message, reply_markup= reply_markup)


  def new_text_message(self, message):
   
    if message == "/start":
      if Controller.turn==0:
        self.chat_id = self.update.message.chat_id
        CommandAnalyzer.playerid_chatid[self.player_id] = self.chat_id
        self.create_player_bot()
      else :
        self.show_message("وسط بازی نمیتونی وارد بشی")
        # update.send_message(update.message.chat_id, text ="وسط بازی نمیتونی وارد شی")
        return 0 
    
    elif message in ["/main_menu","/cancel"]:
      self.request = 0
      self.state = 0 
      self.arguments = []
      self.main_menu()

    else:
      function_names = ["start_production_bot", "buy_product_bot", "show_status_bot", "sell_bot", "sell_to_gamein_bot", "put_into_market_bot"]
      if self.request == 0:
        if message in function_names:
          func = getattr(self, message)
          self.request = message
          func(message)
        else:
          self.show_message("متوجه نشدم")
      else:
          func = getattr(self, self.request)
          func(message)
    

     
  def main_menu(self):

    keyboard = [[InlineKeyboardButton("خرید محصول", callback_data='buy_product_bot')],
                [InlineKeyboardButton("شروع تولید", callback_data='start_production_bot')],
                [InlineKeyboardButton("فروش محصول", callback_data='sell_bot')],
                [InlineKeyboardButton("نمایش وضعیت", callback_data='show_status_bot')]]
    reply_markup = InlineKeyboardMarkup(keyboard)

    self.state = 0 
    self.arguments =[] 
    self.request = 0

    self.show_message("منوی اصلی ",
                              reply_markup=reply_markup)
    
  def create_player_bot(self):
    return_message = Controller.create_player(self.player_id)
    self.show_message(return_message)
    self.main_menu()
    # update.send_message(update.message.chat_id, text =return_message)

  def show_status_bot(self, message):
    status = Controller.show_status(self.player_id)
    self.edit_message_text("بودجه : "+  str(status["cash"])  +"\n"+"موجودی  :" + str(status["inventory"]) +"\n"+ "ظرفیت آزاد انبار :  " + str(status["cap"]))
    self.state = 0 
    self.arguments =[] 
    self.request = 0
    self.main_menu()


  def buy_product_bot(self, message):
    if self.state == 0:
      self.arguments.append(self.player_id)
      keyboard = [[InlineKeyboardButton(i , callback_data=i)] for i in Product.products_volume.index[0:-2]]
      reply_markup = InlineKeyboardMarkup(keyboard)
      self.edit_message_text("چی می خوای بخری؟", reply_markup = reply_markup)
      self.state += 1
    elif self.state == 1:
      vitrin = Controller.show_product_market_info()
      vitrin = vitrin[vitrin.product_name == message]
      keyboard = [[InlineKeyboardButton("فروشنده : " + row.seller_name + " " + ": قیمت " +str(row.price) + " موجودی : " + str(row.amount) , callback_data=str(row.id))] for i, row in vitrin.iterrows()]
      reply_markup = InlineKeyboardMarkup(keyboard)
      self.edit_message_text(" انتخاب کن از کدام یک میخوای " + message + " بخری ", reply_markup = reply_markup)
      self.state += 1
  
    elif self.state == 2:
      self.arguments.append(int (message)) #vitrin id 
      self.edit_message_text("خب... چند تا ازش می خوای بخری؟")
      self.state += 1
    elif self.state == 3:
      self.arguments.append(int (message))
      self.show_message(Controller.buy_product(*self.arguments))
      self.state = 0
      self.arguments = []
      self.request = 0
      self.main_menu()

  def start_production_bot(self,message):
    if self.state == 0 :
      self.arguments.append(self.player_id)
      product_name=list(Controller.player_objects[self.player_id].land_objects.values())[0].machine.product_name
      self.arguments.append(product_name)
      self.edit_message_text(" چه تعداد از " + product_name + " میخوای تولید کنی؟ ")
      self.state +=1 
    elif self.state == 1: 
      self.arguments.append(int (message))
      self.show_message(Controller.start_production(*self.arguments))
      self.state = 0 
      self.arguments =[] 
      self.request = 0
      self.main_menu()
  def sell_bot(self, message):
      keyboard = [[InlineKeyboardButton("قرار دادن در ویترین", callback_data= "put_into_market_bot")], 
                  [InlineKeyboardButton("فروش به گیمین", callback_data= "sell_to_gamein_bot")]]
      reply_markup = InlineKeyboardMarkup(keyboard)
      self.edit_message_text("چطور میخوای بفروشی؟ ", reply_markup = reply_markup)
      self.state = 0 
      self.arguments =[] 
      self.request = 0
  def put_into_market_bot(self, message):
    if self.state == 0 :
      self.arguments.append(self.player_id)
      product_name=list(Controller.player_objects[self.player_id].land_objects.values())[0].machine.product_name
      self.arguments.append(product_name)
      self.edit_message_text(" چه تعداد از " + product_name + "  رو میخوای بفروشی؟  \n موجودی :  " )
      self.state +=1 
    elif self.state == 1: 
      self.arguments.append(int (message))
      self.show_message(" به چه قیمتی ؟ ")
      self.state +=1
    elif self.state == 2 :
      self.arguments.append(int (message))
      self.show_message(Controller.put_into_market(*self.arguments))
      self.state = 0 
      self.arguments =[] 
      self.request = 0
      self.main_menu()
    
  def sell_into_gamein_bot(self, message):
    if self.state == 0 :
      self.arguments.append(self.player_id)
      product_name=list(Controller.player_objects[self.player_id].land_objects.values())[0].machine.product_name
      self.arguments.append(product_name)
      self.edit_message_text(" چه تعداد از " + product_name + "  : رو میخوای بفروشی؟  قیمت فروش هست : " + str(Parameters.gamein_buy_price_df.loc[product_name, "price"]))
      self.state +=1 
    elif self.state == 1: 
      self.arguments.append(int (message))
      self.show_message(Controller.put_into_market(*self.arguments))
      self.state = 0 
      self.arguments =[] 
      self.request = 0
      self.main_menu()
   

  
  

In [ ]:
def main():

    updater = Updater("1321978460:AAFNLXp_sig7IAMrNKjQoSL6ysyh4GmSrAM", use_context=True)
    CommandAnalyzer.bot = Bot("1321978460:AAFNLXp_sig7IAMrNKjQoSL6ysyh4GmSrAM")

    dp = updater.dispatcher
     
    dp.add_handler(CommandHandler(["start","main_menu"] , CommandAnalyzer.new_message))
    dp.add_handler(CallbackQueryHandler(CommandAnalyzer.new_callback, pattern=None))

    dp.add_handler(MessageHandler(Filters.text, CommandAnalyzer.new_message))
    # print("hiiii")
    # dp.add_handler(CommandHandler("hello",CommandAnalyzer.new_command))
    updater.start_polling()

    updater.idle()


  

 
 
if __name__ == '__main__':
  main()

No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/telegram/ext/dispatcher.py", line 340, in process_update
    handler.handle_update(update, self, check, context)
  File "/usr/local/lib/python3.6/dist-packages/telegram/ext/handler.py", line 119, in handle_update
    return self.callback(update, context)
  File "<ipython-input-22-5f60c06dc9aa>", line 29, in new_message
    player_obj.new_text_message(update.message.text)
  File "<ipython-input-22-5f60c06dc9aa>", line 82, in new_text_message
    self.main_menu()
  File "<ipython-input-22-5f60c06dc9aa>", line 112, in main_menu
    reply_markup=reply_markup)
  File "<ipython-input-22-5f60c06dc9aa>", line 48, in show_message
    CommandAnalyzer.bot.send_message(chat_id = self.chat_id, text = text, reply_markup = reply_markup , message_id = self.message_id)
  File "<decorator-gen-124>", line 2, in send_message
  File "/usr/local/lib/python3.6/dist-packages/t

In [ ]:
[print("hi","\n","milad")

hi 
 milad
